In [16]:
import gpytoolbox as gpy
import numpy as np
import trimesh
def yongs_algorithm( points, distances, gradients ):
    '''
    Given a collection of points, where each point has a signed distance value and a gradient.
    For each point, outputs the point -distance units along the gradient direction.
    Parameters:
    points: (N, d) array of point coordinates
        The input points in d-dimensional space.
    distances: (N,) array of signed distance values
        The signed distance values for each point.
    gradients: (N, d) array of gradient vectors
        The gradient vectors at each point.
    '''
    
    # Normalize the gradients to unit vectors
    # norm_gradients = gradients / np.linalg.norm(gradients, axis=1, keepdims=True)
    # Compute the new points by moving along the gradient direction
    new_points = points - (distances[:, np.newaxis] * gradients)
    return new_points


def save_PSR_surface( points, normals, screening_weight = 10.0 ):
    '''
    Runs Poisson Surface Reconstruction to find the surface from the points and normals.
    Parameters:
    points: (N, 3) array of point coordinates
        The input points in 3D space.
    normals: (N, 3) array of normal vectors
        The normal vectors at each point.
    '''
    V,F = gpy.point_cloud_to_mesh( points, normals,
        method='PSR',
        psr_screening_weight=screening_weight,
        psr_outer_boundary_type="Neumann",
        verbose=True
        )
    outpath = "out/" + outbase + "_PSR_surface_" + str(num_points) + ".obj"
    gpy.write_mesh( outpath, V, F )
    print( "Saved PSR surface:", outpath )


In [17]:
# load sdf npz file
data = np.load(r'C:\Users\84432\OneDrive\Documents\code\sdfgradients\out\bunny_sdf_10000.npz')
outbase = "bunny"
points = data['points']  # (N, 3)
sdf_values = data['sdf_values']  # (N,)
gradients = data['gradients']  # (N, 3)
print("Loaded points shape:", points.shape)
print("Loaded sdf_values shape:", sdf_values.shape)
print("Loaded gradients shape:", gradients.shape)
num_points = points.shape[0]

Loaded points shape: (7054, 3)
Loaded sdf_values shape: (7054,)
Loaded gradients shape: (7054, 3)


In [19]:
# Apply Yong's algorithm to find points on the surface
surface_points = yongs_algorithm(points, sdf_values, gradients)
save_PSR_surface( surface_points, gradients )

Saved PSR surface: out/bunny_PSR_surface_7054.obj


In [ ]:
# some sdf data in numpy arrays SDF_POSITIONS, SDF_VALUES
# construct initial mesh
V0, F0 = gpy.icosphere(2)
# call our algorithm
# gpy.reach_for_the_spheres expects an SDF callable; build a fast nearest-neighbor
# interpolant from the sampled `points`->`distances` so we can pass a function.
from scipy.spatial import cKDTree as KDTree
_tree = KDTree(points)
def sdf_callable(q):
    # ensure shape (n,3)
    q = np.asarray(q)
    _, idx = _tree.query(q)
    return sdf_values[idx]
print("sum:", np.sum(sdf_callable(points) - sdf_values))  # verify sdf_callable works

Vr,Fr = gpy.reach_for_the_spheres(points, sdf_callable, V0, F0,
    min_h=0.01,
    remesh_iterations=3,
    verbose=False,
    tol=1e-4)
outpath = "out/" + outbase + "_RFTS_surface_" + str(num_points) + ".obj"
# save the reconstructed mesh
gpy.write_mesh( outpath, Vr, Fr )
print("Saved RFTS/A surface mesh:", outpath)

sum: 0.0


In [20]:
Vr, Fr = gpy.reach_for_the_arcs( points, sdf_values)
outpath = "out/" + outbase + "_RFTA_surface_" + str(num_points) + ".obj"

# save the reconstructed mesh
gpy.write_mesh( outpath, Vr, Fr )
print("Saved RFTS/A surface mesh:", outpath)

Saved RFTS/A surface mesh: out/bunny_RFTA_surface_7054.obj
